# Analiza wykorzystania różnych heurystyk przy rozwiązaniu problemu komiwojażera

Grupa: Amelia Madej, Justyna Sarkowicz, Olga Sieradzan, Weronika Duda i Aleksandra Węgrzyn

## Biblioteki

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import random
import math
 

## Pobór danych 

In [2]:
file_path1 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_48.xlsx"
file_path2 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_76.xlsx"
file_path3 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()

data2 = pd.read_excel(file_path2, index_col=0).to_numpy()

data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Rodzaje ruchów

In [3]:
# Wybieramy dwie pozycje i zmieniamy je miejscami 
def swap_move(path):  
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:]
        new_path[i], new_path[j] = new_path[j], new_path[i]
        yield new_path
 

# Odwracamy segment pomiędzy wybranymi wartościami czyli 
# [a b c d e] -> [a d c b e]
def two_opt_move(path):
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:i] + path[i:j][::-1] + path[j:]
        yield new_path
 
# Wybieramy jeden wierzchołek w ścierzce, usuwamy z jednego miejsca i przerzucamy w inne miejsce
def insertion_move(path):
    n = len(path)
    for i in range(n):
        for j in range(n):
            if i != j:
                new_path = path[:]
                node = new_path.pop(i)
                new_path.insert(j, node)
                yield new_path


## Długość ścierzki

In [4]:
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]


## Algorytm najbliższego sąsiada NN

In [5]:

# Klasyczny algorytm NN

def nearest_neighbor(matrix, start_point=0, max_time=60, random_restart=1):
    n = len(matrix)
    best_path, best_cost = None, float('inf')
    start_time = time.time()

    for _ in range(random_restart):
        # Wybór losowego punktu startowego przy wielu restartach
        current_start_point = np.random.randint(0, n) if random_restart > 1 else start_point

        # Przygotowanie zmiennych dla algorytmu NN
        visited = [False] * n
        path = [current_start_point]
        visited[current_start_point] = True

        # Generowanie ścieżki metodą najbliższego sąsiada
        while len(path) < n:
            last = path[-1]
            next_city = np.argmin([
                matrix[last][j] if not visited[j] else np.inf
                for j in range(n)
            ])
            path.append(next_city)
            visited[next_city] = True

        # Oblicz koszt uzyskanej ścieżki
        cost = calculate_path_cost(matrix, path)

        # Sprawdź, czy znaleziono lepsze rozwiązanie
        if cost < best_cost:
            best_path, best_cost = path, cost

        # Przerwij, jeśli przekroczono limit czasu
        if time.time() - start_time > max_time:
            break

    return best_path, best_cost




# Policzenie łącznej odległości między startem a końcem                



# Optymalizacja rozwiązania 
# Argumenty: matrix odległości / dotychczasowe rozwiązanie / ['swap', '2-opt', 'insertion'] / maksymalna liczba iteracji
def local_search(matrix, initial_path, move_generators, max_iterations):
    best_path = initial_path
    best_cost = calculate_path_cost(matrix, best_path)
 
    iteration = 0
    improved = True
    while improved and iteration < max_iterations:
        improved = False
        for move_generator in move_generators:
            for new_path in move_generator(best_path):
                new_cost = calculate_path_cost(matrix, new_path)
                if new_cost < best_cost:
                    best_path, best_cost = new_path, new_cost
                    improved = True
                    break  
            if improved:
                break
        iteration += 1
 
    return best_path, best_cost

# Ostateczna funkcja 

#  Parametry:
# file_path (str): Ścieżka do pliku Excel zawierającego macierz odległości.
# start_point (int): Miasto początkowe dla algorytmu.
# max_time (int): Maksymalny czas dozwolony na obliczenia (w sekundach).
# move_types (list): Lista typów ruchów dla wyszukiwania lokalnego (np. ['swap', '2-opt', 'insertion']).
# random_restart (int): Liczba losowych restartów w celu wyjścia poza lokalne minima, 
#                       przy każdej iteracji wybieramy nowy start_point a nie cały czas ten sam co ustawiony, jeżeli  random_restart > 1
# max_iterations (int): Maksymalna liczba iteracji dla wyszukiwania lokalnego.
 
# Zwraca:
# best_path (list): Znaleziono optymalną ścieżkę.
# best_cost (float): Długość optymalnej ścieżki.

# Jeżeli move_types=None to używamy wszytskich sposobów do optymalizacji

def solve_tsp(matrix, start_point=0, max_time=60, move_types=None, random_restart=1, max_iterations=100):
   
    
    move_generators = []
    if move_types is None:
        move_types = ['swap', '2-opt', 'insertion']  
    if 'swap' in move_types:
        move_generators.append(swap_move)
    if '2-opt' in move_types:
        move_generators.append(two_opt_move)
    if 'insertion' in move_types:
        move_generators.append(insertion_move)
 
    best_path, best_cost = None, float('inf')
    start_time = time.time()

    # Iteracje z różnym startem
    for _ in range(random_restart):
        # wybranie punktu startującego 
        initial_start_point = np.random.randint(0, len(matrix)) if random_restart > 1 else start_point
 
        # Początkowo generujemy rożwiązanie za pomocą klasycznego NN
        initial_path, initial_cost = nearest_neighbor(matrix, start_point=initial_start_point)
 
        #print(f"Initial NN path (start {initial_start_point}): {initial_path} with cost: {initial_cost}")
 
        # Potem przeprowadzamy optymalizacje z pomocą innych parametrów
        current_path, current_cost = local_search(matrix, initial_path, move_generators, max_iterations)
 
        # Aktualizacja najlepszego wyniku
        if current_cost < best_cost:
            best_path, best_cost = current_path, current_cost
        
 
        # Zabezpieczenie przed przekroczeniem czasu 
        if time.time() - start_time > max_time:
            break
 
    return best_path, best_cost
 

# przykład użycia 
best_path, best_cost = solve_tsp(
    data1, 
    start_point=0, 
    max_time=60, 
    move_types='swap', 
    random_restart=1, 
    max_iterations=150
)


### Generowanie rozwiązań 

Za podstawowe dane przyjmujemy :

* start_ point = 0 

* max_time = 500

* random_restart = 1 

* max_itaration = 150 

Nastepnie badane są wpływy poszczególnych parametrów na każdą metodę 

In [6]:
# Klasyczna metoda NN 
classic_p_1, classic_c_1 = nearest_neighbor(data1, start_point=0 , max_time=500, random_restart=1)


In [7]:
classic_p_2, classic_c_2 = nearest_neighbor(data2, start_point=0, max_time=500, random_restart=1)


In [8]:
classic_p_3, classic_c_3 = nearest_neighbor(data3, start_point=0, max_time=500, random_restart=1)

In [9]:
# Move type = 'swap'

swap_p_1, swap_c_1 = solve_tsp(
    data1, 
    start_point=0, 
    max_time=500, 
    move_types='swap', 
    random_restart=1, 
    max_iterations=150
)



In [10]:
swap_p_2, swap_c_2 = solve_tsp(
    data2, 
    start_point=0, 
    max_time=500, 
    move_types='swap', 
    random_restart=1, 
    max_iterations=150
)



In [11]:
swap_p_3, swap_c_3 = solve_tsp(
    data3, 
    start_point=0, 
    max_time=500, 
    move_types='swap', 
    random_restart=1, 
    max_iterations=150
)

In [12]:
# Move typ = "2 opt"

opt_p_1, opt_c_1 = solve_tsp(
    data1, 
    start_point=0, 
    max_time=500, 
    move_types= '2-opt', 
    random_restart=1, 
    max_iterations=150
)



In [13]:
opt_p_2, opt_c_2 = solve_tsp(
    data2, 
    start_point=0, 
    max_time=500, 
    move_types= '2-opt', 
    random_restart=1, 
    max_iterations=150
)



In [14]:
opt_p_3, opt_c_3 = solve_tsp(
    data3, 
    start_point=0, 
    max_time=500, 
    move_types= '2-opt', 
    random_restart=1, 
    max_iterations=150
)

In [15]:
# Move typ = "insertion"

ins_p_1, ins_c_1 = solve_tsp(
    data1, 
    start_point=0, 
    max_time=500, 
    move_types= 'insertion', 
    random_restart=1, 
    max_iterations=150
)



In [16]:
ins_p_2, ins_c_2 = solve_tsp(
    data2, 
    start_point=0, 
    max_time=500, 
    move_types= 'insertion', 
    random_restart=1, 
    max_iterations=150
)



In [17]:
ins_p_3, ins_c_3 = solve_tsp(
    data3, 
    start_point=0, 
    max_time=500, 
    move_types= 'insertion', 
    random_restart=1, 
    max_iterations=150
)

Zebranie wyników z podstawowymi parametrami 

In [18]:
W1 =  {
    "Długość ścieżki": [classic_c_1, swap_c_1, opt_c_1, ins_c_1]
}

c1 = pd.DataFrame(data = W1)
c1.index = ["Klasyczna","Swap" , "2-opt", "Insertion"]

W2 =  {
    "Długość ścieżki": [classic_c_2, swap_c_2, opt_c_2, ins_c_2]
}

c2 = pd.DataFrame(data = W2)
c2.index = ["Klasyczna","Swap" , "2-opt", "Insertion"]

W3 =  {
    "Długość ścieżki": [classic_c_3, swap_c_3, opt_c_3, ins_c_3]
}

c3 = pd.DataFrame(data = W3)
c3.index = ["Klasyczna","Swap" , "2-opt", "Insertion"]

Badanie wpływu parametru MAX ITTERATIONS

In [20]:
# Stałe parametry
start_point = 0
max_time = 500
random_restart = 1

# DANE NR 1 
results = []
for max_iterations in range(1, 301):
    _, best_cost = solve_tsp(data1, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})


itterration1 = pd.DataFrame(results)


# DANE NR 2
results = []
for max_iterations in range(1, 301):
    _, best_cost = solve_tsp(data2, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})

# Tworzenie ramki danych
itterration2 = pd.DataFrame(results)


# DANE NR 3
results = []
for max_iterations in range(1, 301):
    _, best_cost = solve_tsp(data3, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})

# Tworzenie ramki danych
itterration3 = pd.DataFrame(results)

Badanie wpływu parametru random restart

In [21]:
# Stałe parametry
start_point = 0
max_time = 500
max_iterations = 150

# DANE NR 1 
results = []
for random_restart in range(1, 31):
    _, best_cost = solve_tsp(data1, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": random_restart, "WYNIK": best_cost})


restar1 = pd.DataFrame(results)


# DANE NR 2
results = []
for random_restart in range(1, 31):
    _, best_cost = solve_tsp(data2, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": random_restart, "WYNIK": best_cost})

# Tworzenie ramki danych
restart2 = pd.DataFrame(results)


# DANE NR 3
results = []
for random_restart in range(1, 31):
    _, best_cost = solve_tsp(data3, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": random_restart, "WYNIK": best_cost})

# Tworzenie ramki danych
restart3 = pd.DataFrame(results)

Wystartowanie z 4 różnych losowych puntktów startowych

In [22]:
# Stałe parametry
start_points = [np.random.randint(0, len(data1)), np.random.randint(0, len(data1)), np.random.randint(0, len(data1)), np.random.randint(0, len(data1))]
max_time = 500
max_iterations = 150
random_restart = 1

# DANE NR 1 
results = []
for start_point in start_points:
    _, best_cost = solve_tsp(data1, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": start_point, "WYNIK": best_cost})


point1 = pd.DataFrame(results)

start_points = [np.random.randint(0, len(data2)), np.random.randint(0, len(data2)), np.random.randint(0, len(data2)), np.random.randint(0, len(data2))]

# DANE NR 2
results = []
for start_point in start_points:
    _, best_cost = solve_tsp(data2, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": start_point, "WYNIK": best_cost})

# Tworzenie ramki danych
point2 = pd.DataFrame(results)

start_points = [np.random.randint(0, len(data3)), np.random.randint(0, len(data3)), np.random.randint(0, len(data3)), np.random.randint(0, len(data3))]
# DANE NR 3
results = []
for start_point in start_points:
    _, best_cost = solve_tsp(data3, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": start_point, "WYNIK": best_cost})

# Tworzenie ramki danych
point3 = pd.DataFrame(results)

Zapis wyników w pliku EXCEL

In [23]:
res = {
    "Porównanie_metod+1": c1,
    "max_iterations_1": itterration1,
    "Restart_1": restar1,
    "Point_1" : point1,
    "Porównanie_metod_2": c2,
    "max_iterations_2": itterration2,
    "Restart_2": restart2,
    "Point_2" : point2,
    "Porównanie_metod_3": c3,
    "max_iterations_3": itterration3,
    "Restart_3": restart3,
    "Point_3" : point3,
    
}

# Ścieżka do pliku Excel
file_name = "NN.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in res.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")

Wyniki zostały zapisane w pliku NN.xlsx


## Algorytm wspinaczki z multistartem (IHC)

In [24]:
def local_search(route, distance_matrix, neighborhood_type="swap"):
  

    # Mapowanie typów sąsiedztwa na funkcje
    neighborhood_functions = {
        "swap": swap_move,
        "two_opt": two_opt_move,
        "insertion": insertion_move
    }
    
    if neighborhood_type not in neighborhood_functions:
        raise ValueError(f"Nieznany typ sąsiedztwa: {neighborhood_type}")

    neighbor_func = neighborhood_functions[neighborhood_type]

    n = len(route)
    best_distance = calculate_path_cost( distance_matrix, route)
    improved = True

    while improved:
        improved = False
        for neighbor in neighbor_func(route):
            new_distance = calculate_path_cost( distance_matrix, neighbor)
            if new_distance < best_distance:
                route = neighbor
                best_distance = new_distance
                improved = True
                break  # Skok do następnego sąsiedztwa

    return route, best_distance



    # Algorytm wspinaczki z multistartem dla problemu TSP z kryterium czasowym.
    # Jeśli neighborhood_type=None, działa jak klasyczny IHC bez optymalizacji.
    # distance_matrix: Macierz odległości między miastami.
    # num_starts: Maksymalna liczba startów (iteracji).
    # neighborhood_type: Typ sąsiedztwa ("swap", "two_opt", "insertion").
    # time_limit: Maksymalny czas działania algorytmu w sekundach.
   
def iterated_hill_climbing(distance_matrix, num_starts=100, neighborhood_type="swap", time_limit=None):
  
    n = len(distance_matrix)
    best_route = None
    best_distance = float('inf')

    start_time = time.time()  # Zapisz czas rozpoczęcia

    for start in range(num_starts):
        # Sprawdzenie limitu czasu
        if time_limit and (time.time() - start_time > time_limit):
            print(f"Przekroczono limit czasu ({time_limit} s).")
            break

        # Losowe rozwiązanie początkowe
        initial_route = list(range(n))
        random.shuffle(initial_route)

        # Lokalna wspinaczka lub brak optymalizacji
        local_route, local_distance = local_search(
            initial_route, distance_matrix, neighborhood_type
        )

        # Aktualizacja najlepszego rozwiązania
        if local_distance < best_distance:
            best_route = local_route
            best_distance = local_distance

    return best_route, best_distance

### Generowanie wyników 

Za podstawowe dane przyjmujemy :

* time_limit = 1000

* num_starts = 10 000 

Nastepnie badane są wpływy poszczególnych parametrów na każdą metodę 

In [25]:
# Metoda SWAP
swap_p_1, swap_c_1 = iterated_hill_climbing(data1, num_starts=150,neighborhood_type= "swap", time_limit=1000)

In [26]:
swap_p_2, swap_c_2 = iterated_hill_climbing(data2, num_starts=150,neighborhood_type= "swap", time_limit=1000)

In [27]:
swap_p_3, swap_c_3 = iterated_hill_climbing(data3, num_starts=150,neighborhood_type= "swap", time_limit=1000)

Przekroczono limit czasu (1000 s).


In [28]:
# Metoda Opt-2
opt_p_1, opt_c_1 = iterated_hill_climbing(data1, num_starts=150,neighborhood_type= "two_opt", time_limit=1000)

In [29]:
opt_p_2, opt_c_2 = iterated_hill_climbing(data2, num_starts=150,neighborhood_type= "two_opt", time_limit=1000)

In [30]:
opt_p_3, opt_c_3 = iterated_hill_climbing(data3, num_starts=150,neighborhood_type= "two_opt", time_limit=1000)

Przekroczono limit czasu (1000 s).


In [31]:
# Metoda insertion

ins_p_1, ins_c_1 = iterated_hill_climbing(data1, num_starts=150,neighborhood_type= "insertion", time_limit=1000)

In [32]:
ins_p_2, ins_c_2 = iterated_hill_climbing(data2, num_starts=150,neighborhood_type= "insertion", time_limit=1000)

In [33]:

ins_p_3, ins_c_3 = iterated_hill_climbing(data3, num_starts=150, neighborhood_type= "insertion", time_limit=1000)

Przekroczono limit czasu (1000 s).


Badanie wpływu parametru num_starts 

In [34]:
# Stałe parametry
time_limit = 1000

# DANE NR 1 
results = []
for num_starts in range(1, 200,  20):
    _, best_cost = iterated_hill_climbing(data1, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
    results.append({"PARAMETR": num_starts, "WYNIK": best_cost})


itterration1 = pd.DataFrame(results)


# DANE NR 2

results = []
for num_starts in range(1, 200,  20):
    _, best_cost = iterated_hill_climbing(data2, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
    results.append({"PARAMETR": num_starts, "WYNIK": best_cost})

# Tworzenie ramki danych
itterration2 = pd.DataFrame(results)


# DANE NR 3

results = []
for num_starts in range(1, 200,  20):
    _, best_cost = iterated_hill_climbing(data3, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
    results.append({"PARAMETR": num_starts, "WYNIK": best_cost})

# Tworzenie ramki danych
itterration3 = pd.DataFrame(results)

Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).


Badanie wpływu parametru time_limit

In [35]:
# # Stałe parametry
# num_starts = 1000000

# # DANE NR 1 
# results = []
# for time_limit in range(0, 1000, 100):
#     _, best_cost = iterated_hill_climbing(data1, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
#     results.append({"PARAMETR": time_limit, "WYNIK": best_cost})


# time1 = pd.DataFrame(results)


# # DANE NR 2

# results = []
# for time_limit in range(0, 1000,  100):
#     _, best_cost = iterated_hill_climbing(data2, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
#     results.append({"PARAMETR": time_limit, "WYNIK": best_cost})

# # Tworzenie ramki danych
# time2 = pd.DataFrame(results)


# # DANE NR 3

# results = []
# for time_limit in range(0, 1000, 100):
#     _, best_cost = iterated_hill_climbing(data3, num_starts=num_starts ,neighborhood_type= "two_opt", time_limit=time_limit)
#     results.append({"PARAMETR": time_limit, "WYNIK": best_cost})

# # Tworzenie ramki danych
# time3 = pd.DataFrame(results)



## Algorytm symulowanego wyżarzania (SA)

In [36]:

def get_neighbor(solution, move_type="swap"):
    """
    Generuje sąsiada w zależności od wybranego typu ruchu, korzystając z funkcji generujących sąsiedztwa.
    
    :param solution: Aktualne rozwiązanie (lista miast).
    :param move_type: Typ ruchu do wygenerowania sąsiada ("swap", "insertion", "two_opt").
    :return: Nowe rozwiązanie (sąsiad).
    """
    import random

    # Mapowanie typów ruchów na funkcje generujące sąsiedztwa
    neighborhood_functions = {
        "swap": swap_move,
        "two_opt": two_opt_move,
        "insertion": insertion_move
    }

    if move_type not in neighborhood_functions:
        raise ValueError(f"Nieznany typ ruchu: {move_type}. Dostępne opcje: 'swap', 'insertion', 'two_opt'.")

    # Generowanie wszystkich sąsiadów za pomocą wybranej funkcji
    neighbors = list(neighborhood_functions[move_type](solution))
    
    # Losowy wybór jednego sąsiada
    return random.choice(neighbors)

def simulated_annealing(
    initial_solution,
    distance_matrix,
    initial_temperature=1000,
    cooling_rate=0.95,
    max_iterations=1000,
    min_temperature=1e-3,
    time_limit=None,
    move_type="swap"  # Wybór typu ruchu
):
    """
    Algorytm wyżarzania dla problemu komiwojażera (TSP).
    
    :param initial_solution: Początkowe rozwiązanie (lista miast).
    :param distance_matrix: Macierz odległości między miastami.
    :param initial_temperature: Temperatura początkowa.
    :param cooling_rate: Współczynnik chłodzenia (wartość w zakresie (0, 1)).
    :param max_iterations: Maksymalna liczba iteracji.
    :param min_temperature: Minimalna temperatura.
    :param time_limit: Maksymalny czas działania algorytmu w sekundach.
    :param move_type: Typ ruchu do generowania sąsiada ("swap", "insertion", "two_opt").
    :return: Najlepsze rozwiązanie i jego koszt.
    """
    def calculate_cost(route):
        """Oblicza koszt trasy."""
        return sum(distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1)) + \
               distance_matrix[route[-1]][route[0]]

    current_solution = initial_solution
    current_cost = calculate_cost(current_solution)
    best_solution = current_solution[:]
    best_cost = current_cost

    temperature = initial_temperature
    start_time = time.time()

    for iteration in range(max_iterations):
        if time_limit and (time.time() - start_time > time_limit):
            print("Przekroczono limit czasu.")
            break

        if temperature < min_temperature:
            print("Temperatura osiągnęła wartość minimalną.")
            break

        # Generowanie sąsiada na podstawie wybranego typu ruchu
        neighbor = get_neighbor(current_solution, move_type)
        neighbor_cost = calculate_cost(neighbor)

        delta = neighbor_cost - current_cost
        acceptance_probability = math.exp(-delta / temperature) if delta > 0 else 1

        if random.random() < acceptance_probability:
            current_solution = neighbor
            current_cost = neighbor_cost

        if current_cost < best_cost:
            best_solution = current_solution[:]
            best_cost = current_cost

        temperature *= cooling_rate

        if iteration % 100 == 0:
            print(f"Iteracja {iteration}, najlepszy koszt: {best_cost}, temperatura: {temperature:.2f}")

    return best_solution, best_cost



In [37]:
if __name__ == "__main__":
    import random

    # Przykładowa macierz odległości
    distance_matrix = [
        [0, 29, 20, 21],
        [29, 0, 15, 17],
        [20, 15, 0, 28],
        [21, 17, 28, 0]
    ]

    # Początkowe rozwiązanie (losowa permutacja miast)
    initial_solution = list(range(len(distance_matrix)))
    random.shuffle(initial_solution)

    # Wywołanie algorytmu z różnymi rodzajami ruchów
    for move_type in ["swap", "insertion", "two_opt"]:
        print(f"\nAlgorytm wyżarzania z ruchem: {move_type}")
        best_solution, best_cost = simulated_annealing(
            initial_solution,
            distance_matrix,
            initial_temperature=1000,
            cooling_rate=0.99,
            max_iterations=10000,
            min_temperature=0.01,
            time_limit=5,
            move_type=move_type
        )
        print(f"Najlepsza trasa ({move_type}): {best_solution}")
        print(f"Najkrótsza odległość: {best_cost}")


Algorytm wyżarzania z ruchem: swap
Iteracja 0, najlepszy koszt: 93, temperatura: 990.00
Iteracja 100, najlepszy koszt: 73, temperatura: 362.37
Iteracja 200, najlepszy koszt: 73, temperatura: 132.64
Iteracja 300, najlepszy koszt: 73, temperatura: 48.55
Iteracja 400, najlepszy koszt: 73, temperatura: 17.77
Iteracja 500, najlepszy koszt: 73, temperatura: 6.50
Iteracja 600, najlepszy koszt: 73, temperatura: 2.38
Iteracja 700, najlepszy koszt: 73, temperatura: 0.87
Iteracja 800, najlepszy koszt: 73, temperatura: 0.32
Iteracja 900, najlepszy koszt: 73, temperatura: 0.12
Iteracja 1000, najlepszy koszt: 73, temperatura: 0.04
Iteracja 1100, najlepszy koszt: 73, temperatura: 0.02
Temperatura osiągnęła wartość minimalną.
Najlepsza trasa (swap): [2, 0, 3, 1]
Najkrótsza odległość: 73

Algorytm wyżarzania z ruchem: insertion
Iteracja 0, najlepszy koszt: 93, temperatura: 990.00
Iteracja 100, najlepszy koszt: 73, temperatura: 362.37
Iteracja 200, najlepszy koszt: 73, temperatura: 132.64
Iteracja 300,

## Algorytm przeszukiwania Tabu 

## Algorytm genetyczny

## Algorytm zachłanny

In [38]:
## Klasyczny algorytm zachłanny 
def tsp_greedy(distance_matrix, start=0):
    n = len(distance_matrix)
    visited = [False] * n
    visited[start] = True
    path = [start]
    total_cost = 0
    current_city = start

    for _ in range(n - 1):
        # Znajdujemy najbliższe miasto którego nie odwiedziliśmy 
        next_city = min(
            (city for city in range(n) if not visited[city]),
            key=lambda city: distance_matrix[current_city][city]
        )
        # Dodajemy odległośc i zaznaczamy ze miasto jest już odwiedzone
        total_cost += distance_matrix[current_city][next_city]
        visited[next_city] = True
        path.append(next_city)
        current_city = next_city

    # Obliczanie końcowej trasy
    total_cost += distance_matrix[current_city][start]
    path.append(start)

    return path, total_cost

## Iteracja algorytmem zachłannym po każdym możliwym punkcie startowym 

def tsp_greedy_all_starts(distance_matrix):
    best_cost = float('inf')
    best_path = None

    for start in range(len(distance_matrix)):
        path, cost = tsp_greedy(distance_matrix, start=start)
        if cost < best_cost:
            best_cost = cost
            best_path = path
            point = start

    return best_cost, best_path, point

In [39]:

## Iteracja algorytmem zachłannym po każdym możliwym punkcie startowym 

def tsp_greedy_all_starts(distance_matrix):
    best_cost = float('inf')
    best_path = None

    for start in range(len(distance_matrix)):
        path, cost = tsp_greedy(distance_matrix, start=start)
        if cost < best_cost:
            best_cost = cost
            best_path = path
            point = start

    return best_cost, best_path, point

In [40]:

## Iteracja algorytmem zachłannym po każdym możliwym punkcie startowym 

def tsp_greedy_all_starts(distance_matrix):
    best_cost = float('inf')
    best_path = None

    for start in range(len(distance_matrix)):
        path, cost = tsp_greedy(distance_matrix, start=start)
        if cost < best_cost:
            best_cost = cost
            best_path = path
            point = start

    return best_cost, best_path, point

## Generowanie rozwiazań

In [41]:
# Klasyczny ( od punktu 1 )
classic_p_1, classic_c_1 = tsp_greedy(data1)
classic_p_2, classic_c_2 = tsp_greedy(data2)
classic_p_3, classic_c_3 = tsp_greedy(data3)

In [42]:
# Start we wszytskich punktach 
all_p_1, all_c_1, all_point_1= tsp_greedy_all_starts(data1)
all_p_2, all_c_2, all_point_2 = tsp_greedy_all_starts(data2)
all_p_3, all_c_3, all_point_3 = tsp_greedy_all_starts(data3)

Zebranie wyników oraz zapis w pliku EXCEL 

In [43]:
W1 =  {
    "Długość ścieżki": [classic_c_1,classic_c_2,classic_c_3 ]
}

p1 = pd.DataFrame(data = W1)
p1.index = ["Dane 1","Dane 2" , "Dane 3", ]

W2 =  {
    "Długość ścieżki": [all_p_1, all_p_2, all_p_3],
    "Punkt": [all_point_1,all_point_2, all_point_3]
}

p2 = pd.DataFrame(data = W2)
p2.index = ["Dane 1","Dane 2" , "Dane 3"]

resu = {
    "Klasyka": p1,
    "Wszytskie":p2    
}

# Ścieżka do pliku Excel
file_name = "Zachłanny.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in resu.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")

Wyniki zostały zapisane w pliku Zachłanny.xlsx
